In [31]:
import tensorflow as tf

In [32]:
class NaiveDense:
    def __init__(self, input_size, output_size, activation):
        self.activation = activation
        w_shape = (input_size, output_size)
        w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
        self.W = tf.Variable(w_initial_value)

        b_shape = (output_size,)

        b_initial_value = tf.zeros(b_shape)
        self.b = tf.Variable(b_initial_value)

    def __call__(self, inputs):
        return self.activation(tf.matmul(inputs, self.W) + self.b)
    

    @property
    def weights(self):
        return [self.W, self.b]
    
class NaiveSequential:
    def __init__(self, layers):
        self.layers = layers
    
    def __call__(self, inputs):
        x = inputs 
        for layer in self.layers:
            x = layer(x)
        return x
        
    @property
    def weights(self):
        weights = []
        for layer in self.layers:
            weights += layer.weights
        return weights

In [33]:
model = NaiveSequential([
    NaiveDense(input_size=28 * 28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)
])

assert len(model.weights) == 4

In [34]:
import math

class BatchGenerator:
    def __init__(self, images, labels, batch_size=128):
        assert len(images) == len(labels)

        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = math.ceil(len(images) / batch_size)

    def next(self):
        images = self.images[self.index : self.index + self.batch_size]
        labels = self.labels[self.index : self.index + self.batch_size]

        self.index += self.batch_size
        return images, labels

In [35]:
def one_training_step(model, images_batch, labels_batch):
    with tf.GradientTape() as tape:
        predictions = model(images_batch)

        per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(labels_batch, predictions)
        average_loss = tf.reduce_mean(per_sample_losses)

        gradients = tape.gradient(average_loss, model.weights)
        update_weights(gradients, model.weights)
        

        return average_loss
    
learning_rate = 1e-3

# def update_weights(gradients, weights):
#     for g, w in zip(gradients, weights):
#         w.assign_sub(g * learning_rate)

In [36]:
from tensorflow.keras import optimizers

optimizer = optimizers.SGD(learning_rate=1e-3)

def update_weights(gradients, weights):
    optimizer.apply_gradients(zip(gradients, weights))
 

In [37]:
def fit(model, images, labels, epochs, batch_size=128):
    for epoch_counter in range(epochs):
        print("Epoch ", epoch_counter)
        batch_generator = BatchGenerator(images, labels)
        for batch_counter in range(batch_generator.num_batches):
            images_batch, labels_batch = batch_generator.next()
            loss = one_training_step(model, images_batch, labels_batch)
            if batch_counter % 100 == 0:
                print(f"loss at batch {batch_counter}: {loss:.2f}")


In [38]:
import matplotlib.pyplot as plt
from keras import datasets

(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

In [39]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

In [40]:
fit(model, train_images, train_labels, epochs=10, batch_size=128)

Epoch  0
loss at batch 0: 6.34
loss at batch 100: 2.23
loss at batch 200: 2.20
loss at batch 300: 2.08
loss at batch 400: 2.19
Epoch  1
loss at batch 0: 1.89
loss at batch 100: 1.88
loss at batch 200: 1.83
loss at batch 300: 1.72
loss at batch 400: 1.81
Epoch  2
loss at batch 0: 1.57
loss at batch 100: 1.58
loss at batch 200: 1.51
loss at batch 300: 1.43
loss at batch 400: 1.51
Epoch  3
loss at batch 0: 1.32
loss at batch 100: 1.35
loss at batch 200: 1.24
loss at batch 300: 1.21
loss at batch 400: 1.28
Epoch  4
loss at batch 0: 1.12
loss at batch 100: 1.17
loss at batch 200: 1.04
loss at batch 300: 1.05
loss at batch 400: 1.11
Epoch  5
loss at batch 0: 0.98
loss at batch 100: 1.03
loss at batch 200: 0.90
loss at batch 300: 0.93
loss at batch 400: 0.99
Epoch  6
loss at batch 0: 0.87
loss at batch 100: 0.92
loss at batch 200: 0.80
loss at batch 300: 0.84
loss at batch 400: 0.90
Epoch  7
loss at batch 0: 0.79
loss at batch 100: 0.84
loss at batch 200: 0.72
loss at batch 300: 0.77
loss at 

In [41]:
import numpy as np

predictions = model(test_images)
predictions = predictions.numpy()
predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels

print(f"accuracy: {matches.mean():.2f}")


accuracy: 0.82
